# Create and populate the database

In [ ]:
# # Import standard libraries
# import os
# import re
# import json
# import codecs
# import requests
import dill
# from urllib.request import urlopen

# # Import third-party libraries
# import geopandas as gpd
# from geoalchemy2 import Geometry
# import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, MetaData, event, Table, text, LargeBinary
from sqlalchemy.dialects.sqlite import insert
from sqlalchemy.orm import sessionmaker
# from sqlalchemy.event import listen
# from sqlalchemy.engine import Engine
# from sqlalchemy.ext.declarative import declarative_base

# import sqlite3
# from sqlite3 import dbapi2 as sqlite

# import fiona
# from fiona.crs import from_epsg

from src.helpers import *
from src.dbutils import *
from src.ORMutils import *
from src.models import *
from src.geo import *
from src.pdfutils import *

In [ ]:
# Load the environment
with open("environment_data/table_dicts.pkl", "rb") as f:
    env = dill.load(f)

# Restore the environment
globals().update(env)

In [ ]:
# Load the environment
with open("environment_data/select.pkl", "rb") as f:
    env = dill.load(f)

# Restore the environment
globals().update(env)

In [ ]:
for name, dataset in datasets.items():
    print(dataset.short_name)

* ### Create the database engine that will be used throughout the rest of the notebook.

In [ ]:
engine = create_engine(f'{SQLITE_PATH}?check_same_thread=False', echo=False)

SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)

* ### Configure the database

In [ ]:
@event.listens_for(engine, "connect")
def load_spatialite(dbapi_conn, connection_record):
    print("Loading SpatiaLite extension")
    dbapi_conn.enable_load_extension(True)
    dbapi_conn.load_extension("mod_spatialite")
    dbapi_conn.enable_load_extension(False)


with engine.connect() as conn:
    print("Connection established")
    result = conn.execute(text("SELECT spatialite_version()"))
    spatialite_version = result.fetchone()
    print(f"SpatiaLite version: {spatialite_version[0]}")

# Enable WAL mode
with engine.connect() as conn:
    conn.execute(text("PRAGMA journal_mode=WAL"))

# Initialize spatial metadata if not already present
with engine.connect() as conn:
    conn.execute(text("SELECT InitSpatialMetaData(1)"))

### Manually create the borough codes lookup table
* These are standardized and available many places, however I could not find a single official source of record to programatically get them from, since there are only five of them, I enter them manually.

In [ ]:
borough_codes = {'Manhattan' : 1,
'Bronx' : 2,
'Brooklyn' : 3,
'Queens' : 4,
'Staten Island' : 5}

* Create the lookup table.

In [ ]:
metadata = MetaData()
metadata.reflect(bind=engine)

def create_lookup_table_simple(engine=engine, metadata=metadata, lookup_table_name='new_lookup_table', lookup_column_name='name'):
    lookup_table = Table(
        lookup_table_name,
        metadata,
        Column('id', Integer, primary_key=True, autoincrement=False),
        Column(lookup_column_name, String, unique=True, nullable=False, default="NO DATA"),
        extend_existing = True
    )
    if table_exists(engine, lookup_table_name):
        print("Table exists")
    else:
        lookup_table.create(engine)
    return lookup_table

* Populate the table with the borough codes.

In [ ]:
borough_lookup_table = create_lookup_table_simple(engine=engine, metadata=metadata, lookup_table_name='boroughs', lookup_column_name='borough')

with engine.connect() as connection:
    for key,value in borough_codes.items():
        stmt = insert(borough_lookup_table).values(id = value, borough = key).on_conflict_do_nothing()
        connection.execute(stmt)
    connection.commit()

* ### Create lookup tables variables identified as categorical and for which definitions were extracted from the metadata in the previous notebook.

In [ ]:
lookups = [col for col in column_customizations if col.definitions ]

for table in lookups:
    print(table)
    lookup_table_name=table.new_name.title()
    with engine.connect() as connection:
        lookup_table = create_lookup_table(engine=engine, lookup_table_name=lookup_table_name, text_column_name='name_or_code')
        if lookup_table is None:
            print(f"Table {lookup_table_name} was not properly retrieved or created. Perhaps it exists? Skipping for now..")
            continue
        for definition in table.definitions:
            print(definition)
            if len(definition) == 2:
                try:
                    stmt = insert(lookup_table).values(id=int(definition[0]), name_or_code=definition[1]).on_conflict_do_nothing()
                except ValueError:
                    stmt = insert(lookup_table).values(name_or_code=definition[0], info=definition[1]).on_conflict_do_nothing()
            elif len(definition) == 3:
                try:
                    stmt = insert(lookup_table).values(id=int(definition[0]), name_or_code=definition[1], info=definition[2]).on_conflict_do_nothing()
                except ValueError:
                    stmt = insert(lookup_table).values(name_or_code=definition[0], info=definition[1], info=definition[2]).on_conflict_do_nothing()
            else:
                raise ValueError("Was only expecting two or three columns")
            connection.execute(stmt)
        connection.commit()
